# Image Detector

Instructions: Set the IMG_FOLDER, IMG_FILE, and CHECKPOINT to perform object detection on a specific image from a checkpoint of the model

In [ ]:
IMG_FOLDER = 'test'
IMG_FILE = 'screenshot_02a9.png'
CHECKPOINT = 'ckpt-3'
CUSTOM_MODEL_NAME = 'dgtlvsn_ssd_mobnet_640_v2'
BLUR = True

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
LABEL_MAP_NAME = 'label_map.pbtxt'
paths = {
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGES_PATH': os.path.join('Tensorflow', 'workspace','images'),
}

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], CHECKPOINT)).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
IMAGE_PATH = os.path.join(paths['IMAGES_PATH'], IMG_FOLDER, IMG_FILE)

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=10,
            min_score_thresh=.8,
            agnostic_mode=False)
if BLUR:
    for i, (box, score) in enumerate(zip(detections['detection_boxes'], detections['detection_scores'])):
        if score > 0.8:
            ymin, xmin, ymax, xmax = box
            h, w, _ = image_np.shape
            y_min = int(ymin * h)
            y_max = int(ymax * h)
            x_min = int(xmin * w)
            x_max = int(xmax * w)
        
            # Extract the region of interest within the detection box
            roi = image_np[y_min:y_max, x_min:x_max]
        
            # Apply Gaussian blur to the ROI
            blurred_roi = cv2.GaussianBlur(roi, (45, 45), 0)  # You can adjust the kernel size
        
            # Replace the original ROI with the blurred ROI
            image_np_with_detections[y_min:y_max, x_min:x_max] = blurred_roi


plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()